<a href="https://colab.research.google.com/github/SEOYUNJE/Endoscope-Object-Detection/blob/main/Weighted-Boxes-Fusion/GastroScopy/WBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q opendatasets

import opendatasets as od

od.download('https://www.kaggle.com/datasets/seoyunje/gastroscopy-submission')
od.download('https://www.kaggle.com/datasets/seoyunje/gastroscopy-256x256-resized-png')
od.download('https://www.kaggle.com/datasets/msyu78/gastroscopy-meta')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: seoyunje
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/seoyunje/gastroscopy-submission


100%|██████████| 4.73M/4.73M [00:00<00:00, 38.5MB/s]

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: seoyunje
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/seoyunje/gastroscopy-256x256-resized-png


100%|██████████| 177M/177M [00:01<00:00, 121MB/s]



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: seoyunje
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/msyu78/gastroscopy-meta


100%|██████████| 802k/802k [00:00<00:00, 98.7MB/s]

## Decoding PredictionString
___

In [75]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
import cv2

**Utils Function**

In [3]:
def string_decoder(predict_string):
    s = predict_string.split(' ')
    class_id = [x for x in (s[0::6])]
    score = [x for x in (s[1::6])]
    x_min = [x for x in (s[2::6])]
    y_min = [x for x in (s[3::6])]
    x_max = [x for x in (s[4::6])]
    y_max = [x for x in (s[5::6])]

    return class_id, score, x_min, y_min, x_max, y_max

In [4]:
def format_pred(boxes, scores, labels):
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.float32)
        pred_strings.append(f"{label} {score} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)

### 1. Yolo Submission

In [5]:
yolo_sub = pd.read_csv('/content/gastroscopy-submission/yolo_submission.csv')

In [6]:
yolo_list = []

for i, row in tqdm(yolo_sub.iterrows(), total=len(yolo_sub)):
    image_id = row['image_id']
    string_decoded = string_decoder(row['PredictionString'])

    for  (class_id, score, xmin, ymin, xmax, ymax) in zip(*string_decoded):
        yolo_list.append(
            {
                'image_id': image_id,
                'lesion_label': int(class_id),
                'score': float(score),
                'x_min': float(xmin),
                'y_min': float(ymin),
                'x_max': float(xmax),
                'y_max': float(ymax),
                }
        )

yolo_submission = pd.DataFrame(yolo_list, columns=['image_id', 'lesion_label', 'score','x_min','y_min', 'x_max', 'y_max'])

100%|██████████| 400/400 [00:00<00:00, 412.05it/s]


In [7]:
print('Shape of DataFrame: ', yolo_submission.shape)
print(display(yolo_submission))

Shape of DataFrame:  (200000, 7)


,image_id,lesion_label,score,x_min,y_min,x_max,y_max
0,1_1_00030.png,0,0.4148,986.8,1254.8,1405.6,1711.7
1,1_1_00030.png,0,0.2025,1097.8,1304.9,1350.4,1669.0
2,1_1_00030.png,0,0.1695,1049.0,1293.8,1371.7,1749.5
3,1_1_00030.png,0,0.1593,529.2,589.3,644.4,718.0
4,1_1_00030.png,0,0.1441,1024.2,1235.4,1378.9,1630.2
...,...,...,...,...,...,...,...
199995,1_3_09997.png,0,0.0000,1676.2,1340.7,1930.7,1672.8
199996,1_3_09997.png,0,0.0000,855.0,897.0,1543.3,1335.7
199997,1_3_09997.png,0,0.0000,1517.1,143.9,1882.9,391.8
199998,1_3_09997.png,1,0.0000,181.3,144.3,504.7,322.6


None


### 2. Detectron2 Submission

In [8]:
detectron2_sub = pd.read_csv('/content/gastroscopy-submission/detectron2_submission.csv')

In [9]:
detectron2_list = []

for i, row in tqdm(detectron2_sub.iterrows(), total=len(detectron2_sub)):
    image_id = row['image_id']
    string_decoded = string_decoder(row['PredictionString'])

    for  (class_id, score, xmin, ymin, xmax, ymax) in zip(*string_decoded):
        detectron2_list.append(
            {
                'image_id': image_id,
                'lesion_label': int(class_id),
                'score': float(score),
                'x_min': float(xmin),
                'y_min': float(ymin),
                'x_max': float(xmax),
                'y_max': float(ymax),
                }
        )

detectron2_submission = pd.DataFrame(detectron2_list, columns=['image_id', 'lesion_label', 'score','x_min','y_min', 'x_max', 'y_max'])

100%|██████████| 400/400 [00:00<00:00, 632.85it/s]


In [10]:
print('Shape of DataFrame: ', detectron2_submission.shape)
print(display(detectron2_submission))

Shape of DataFrame:  (39997, 7)


,image_id,lesion_label,score,x_min,y_min,x_max,y_max
0,1_1_00030.png,0,0.858911,1119.544678,1281.931396,1298.300781,1594.814697
1,1_1_00030.png,0,0.821695,821.965820,1072.834595,1049.765015,1353.154175
2,1_1_00030.png,0,0.818180,529.171021,580.363708,651.071777,752.823059
3,1_1_00030.png,0,0.806905,1049.580322,1296.518433,1315.405518,1646.039185
4,1_1_00030.png,0,0.702657,1146.296875,1312.744385,1302.452393,1454.972168
...,...,...,...,...,...,...,...
39992,1_3_09997.png,1,0.000672,877.135498,655.416138,1145.073853,896.607910
39993,1_3_09997.png,2,0.000644,1161.219360,1000.442749,1598.245483,1727.570679
39994,1_3_09997.png,0,0.000643,880.915649,648.346191,1141.772827,891.302124
39995,1_3_09997.png,2,0.000642,1394.356934,1283.259644,1940.879883,1816.635864


None


### 3. EfficientDet Submission

In [11]:
effdet_sub = pd.read_csv('/content/gastroscopy-submission/effdet_submission.csv')

In [12]:
effdet_list = []

for i, row in tqdm(effdet_sub.iterrows(), total=len(effdet_sub)):
    image_id = row['image_id']
    string_decoded = string_decoder(row['PredictionString'])

    for  (class_id, score, xmin, ymin, xmax, ymax) in zip(*string_decoded):
        effdet_list.append(
            {
                'image_id': image_id,
                'lesion_label': int(class_id),
                'score': float(score),
                'x_min': float(xmin),
                'y_min': float(ymin),
                'x_max': float(xmax),
                'y_max': float(ymax),
                }
        )

effdet_submission = pd.DataFrame(effdet_list, columns=['image_id', 'lesion_label', 'score','x_min','y_min', 'x_max', 'y_max'])

100%|██████████| 400/400 [00:00<00:00, 435.84it/s]


In [13]:
print('Shape of DataFrame: ', effdet_submission.shape)
print(display(effdet_submission))

Shape of DataFrame:  (40000, 7)


,image_id,lesion_label,score,x_min,y_min,x_max,y_max
0,1_1_00030.png,0,0.617498,970.773438,1222.369873,1384.108887,1669.809814
1,1_1_00030.png,0,0.298233,597.309998,204.998291,775.652405,383.554810
2,1_1_00030.png,0,0.159930,413.295685,503.283203,662.980225,765.230103
3,1_1_00030.png,0,0.151825,356.673523,450.854858,564.194397,661.274902
4,1_1_00030.png,1,0.135376,382.581970,483.293610,508.141663,610.424011
...,...,...,...,...,...,...,...
39995,1_2_05233.png,0,0.016218,1779.818970,1425.876709,2048.000000,1781.247070
39996,1_2_05233.png,2,0.016213,726.055481,1415.148926,2048.000000,2048.000000
39997,1_2_05233.png,1,0.016125,661.229919,439.486084,854.199524,534.231995
39998,1_2_05233.png,2,0.016067,1639.281494,1591.289795,2035.996826,1997.776123


None


## Techniques to Combine Boxes: WBF
___

In [ ]:
!pip install -q ensemble-boxes

In [ ]:
from ensemble_boxes import *

**Setting Model Weight**

In [ ]:
model_weight = {
    'yolo': 1.0,
    'detectron2': 1.0,
    'effdet': 1.0,
}

In [ ]:
test_meta = pd.read_csv('/content/gastroscopy-256x256-resized-png/test_meta.csv')

iou_threshold = 0.5 # Default: 0.55
skip_box_thr = 0.005 # Default: 0.0
conf_type = 'max' # avg, max, box_and_model_avg, absent_model_aware_avg
allows_overflow = False # Default: False
weights = None # Default: None, which means weight == 1 for each model

result_list = []

pbar = tqdm(enumerate(yolo_submission['image_id'].unique()), total=len(yolo_submission['image_id'].unique()), desc = 'Applying NMS')

for i, image_id in pbar:

    height, width = test_meta[test_meta['image_id'] == image_id][['height', 'width']].values[0]

    meta_yolo = yolo_submission[yolo_submission['image_id'] == image_id]
    meta_detectron2 = detectron2_submission[detectron2_submission['image_id'] == image_id]
    meta_effdet = effdet_submission[effdet_submission['image_id'] == image_id]

    boxes_list = []
    scores_list = []
    labels_list = []

    weights = []

    ### yolo submssion
    label = meta_yolo['lesion_label'].tolist(); labels_list.append(label)
    score = meta_yolo['score'].tolist(); scores_list.append(score)

    bbox = meta_yolo[['x_min','y_min','x_max','y_max']].values
    bbox[:,[0,2]] /= width; bbox[:,[1,3]] /= height
    #bbox = np.clip(bbox, 0, 1)
    boxes_list.append(bbox.tolist())

    weights.append(model_weight['yolo'])

    ### detectron2 submssion
    label = meta_detectron2['lesion_label'].tolist(); labels_list.append(label)
    score = meta_detectron2['score'].tolist(); scores_list.append(score)

    bbox = meta_detectron2[['x_min','y_min','x_max','y_max']].values
    bbox[:,[0,2]] /= width; bbox[:,[1,3]] /= height
    #bbox = np.clip(bbox, 0, 1)
    boxes_list.append(bbox.tolist())

    weights.append(model_weight['detectron2'])

    ### effdet submssion
    label = meta_effdet['lesion_label'].tolist(); labels_list.append(label)
    score = meta_effdet['score'].tolist(); scores_list.append(score)

    # Normalized
    bbox = meta_effdet[['x_min','y_min','x_max','y_max']].values
    bbox[:,[0,2]] /= width; bbox[:,[1,3]] /= height
    #bbox = np.clip(bbox, 0, 1)
    boxes_list.append(bbox.tolist())

    weights.append(model_weight['effdet'])

    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr = iou_threshold, skip_box_thr=skip_box_thr, conf_type=conf_type, allows_overflow=allows_overflow)
    boxes = boxes*(width, height, width, height)

    boxes = boxes.tolist(); labels = labels.astype(int).tolist(); scores = scores.astype(float).tolist()

    result = {
        'image_id': image_id,
        'bboxes': boxes,
        'scores': scores,
        'lesion_labels': labels,
        'width': width,
        'height': height
    }
    result_list.append(result)

In [94]:
tmp = pd.DataFrame(result_list, columns=['image_id', 'bboxes', 'scores', 'lesion_labels', 'width', 'height'])

expanded_rows = []

for _, row in tmp.iterrows():
    image_id = row["image_id"]
    bboxes = row["bboxes"]
    scores = row['scores']
    lesion_labels = row["lesion_labels"]
    width = row['width']
    height = row['height']

    for bbox, lesion_label, score in zip(bboxes, lesion_labels, scores):
        expanded_rows.append({
            "image_id": image_id,
            "x_min": bbox[0], # original image
            "y_min": bbox[1], # original image
            "x_max": bbox[2], # original image
            "y_max": bbox[3], # original image
            "lesion_label": lesion_label,
            'score': score,
            "width": width,
            "height": height,
           })

expanded_df = pd.DataFrame(expanded_rows)

# 결과 출력
print(display(expanded_df))
print('=> Bboxes Before WBF: ', len(yolo_submission) + len(detectron2_submission) + len(effdet_submission))
print('=> Bboxes After WBF: ', len(expanded_df))

wbf_df = expanded_df.reset_index(drop=True)

,image_id,x_min,y_min,x_max,y_max,lesion_label,score,width,height
0,1_1_00030.png,1094.976929,1290.671143,1312.863892,1598.183105,0,0.858911,2048,2048
1,1_1_00030.png,820.597534,1066.615967,1051.206299,1350.650391,0,0.821695,2048,2048
2,1_1_00030.png,525.185242,577.807007,651.286926,750.613647,0,0.818180,2048,2048
3,1_1_00030.png,1143.707642,1311.918091,1300.209351,1457.816772,0,0.702657,2048,2048
4,1_1_00030.png,984.426392,1231.899048,1386.612549,1682.191650,0,0.617498,2048,2048
...,...,...,...,...,...,...,...,...,...
45572,1_3_09997.png,709.171204,728.324341,1009.947327,1053.151855,1,0.012358,2048,2048
45573,1_3_09997.png,1080.820679,642.523682,1897.848633,1285.107788,1,0.009504,2048,2048
45574,1_3_09997.png,1530.349365,767.849548,1999.254883,1260.472412,2,0.007565,2048,2048
45575,1_3_09997.png,901.146179,392.751831,1934.575928,1089.588257,2,0.005727,2048,2048


None
=> Bboxes Before WBF:  279997
=> Bboxes After WBF:  45577


**Calculating mAP50, mAP75**

In [20]:
!pip install map-boxes
from map_boxes import mean_average_precision_for_boxes

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [95]:
test_id = test_meta['image_id'].unique()
ann = pd.read_csv('/content/gastroscopy-meta/gastroscopy_data2.csv')
ann = ann[ann['image_id'].isin(test_id)].reset_index(drop=True)

det = wbf_df.copy()

ann = ann[['image_id','lesion_label','x_min','x_max','y_min','y_max']].values

det = det[['image_id','lesion_label', 'score','x_min','x_max','y_min','y_max']].values

mean_ap_50, average_precisions_50 = mean_average_precision_for_boxes(ann, det, iou_threshold=0.5)
mean_ap_75, average_precisions_75 = mean_average_precision_for_boxes(ann, det, iou_threshold=0.75)

Number of files in annotations: 400
Number of files in predictions: 400
Unique classes: 3
Detections length: 400
Annotations length: 400
0                              | 0.570549 |     150
1                              | 0.740530 |     270
2                              | 0.815735 |     231
mAP: 0.708938
Number of files in annotations: 400
Number of files in predictions: 400
Unique classes: 3
Detections length: 400
Annotations length: 400
0                              | 0.278305 |     150
1                              | 0.427691 |     270
2                              | 0.485191 |     231
mAP: 0.397062


### Encoding PredictionString

In [53]:
result_list = []

for i, image_id in tqdm(enumerate(nms_df['image_id'].unique()), total=len(nms_df['image_id'].unique())):
    tmp = nms_df[nms_df['image_id'] == image_id]
    boxes = tmp[['x_min','y_min','x_max','y_max']].values
    scores = tmp['score'].tolist()
    labels = tmp['lesion_label'].tolist()


    result = {
        'image_id': image_id,
        'PredictionString': format_pred(boxes, scores, labels)
    }
    result_list.append(result)

100%|██████████| 400/400 [00:07<00:00, 53.71it/s]


In [54]:
submission_det = pd.DataFrame(result_list, columns=['image_id','PredictionString'])
print(display(submission_det))

,image_id,PredictionString
0,1_1_00030.png,0 0.2863037387530009 1119.544677734375 1281.93...
1,1_1_00033.png,0 0.3286905884742737 550.513671875 1133.995483...
2,1_1_00035.png,0 0.22513226668039957 228.47454833984375 827.3...
3,1_1_00040.png,0 0.3277176022529602 661.5875244140625 1203.64...
4,1_1_00095.png,0 0.33301154772440594 784.8660888671875 494.92...
...,...,...
395,1_3_09783.png,0 0.0159036119778951 0.0 624.8349609375 162.12...
396,1_3_09808.png,0 0.020976975560188293 650.4411010742188 948.3...
397,1_3_09826.png,0 0.007588876411318779 450.22406005859375 119....
398,1_3_09872.png,0 0.006160684550801913 307.8858337402344 0.0 8...


None
